# Data Reformation $\,\, \tiny\text{Analysis | NASDAQ & NYSE (Stock Market)}$

<img src="banner lab 1.jpg" align=left>

### About the Business Situation & Data
___

The dataset that used for this analysis is called ** $\,$ `dataset tech stocks.xlsx` $\,$ **. 

### Apparatus
___

In [1]:
options(warn=-1)

# Load some required functions
library(rJava,      verbose=FALSE, warn.conflicts=FALSE, quietly=TRUE)
library(xlsxjars,   verbose=FALSE, warn.conflicts=FALSE, quietly=TRUE)
library(xlsx,       verbose=FALSE, warn.conflicts=FALSE, quietly=TRUE) # Also, ensure Java version (32-bit or 64-bit) matches R kernel
library(lubridate,  verbose=FALSE, warn.conflicts=FALSE, quietly=TRUE)
library(reshape2,   verbose=FALSE, warn.conflicts=FALSE, quietly=TRUE)
library(ggplot2,    verbose=FALSE, warn.conflicts=FALSE, quietly=TRUE)

# Set some visualization formatting defaults
options(repr.plot.width=6, repr.plot.height=4)
theme_update(plot.title=element_text(size=10, hjust=0.5), plot.subtitle=element_text(size=8, face="italic", hjust=0.5), axis.title=element_text(size=7), axis.text=element_text(size=7), strip.text=element_text(size=7), strip.text.y=element_text(angle=90), legend.title=element_blank(), legend.text=element_text(size=7))

ERROR: Error in library(rJava, verbose = FALSE, warn.conflicts = FALSE, quietly = TRUE): there is no package called 'rJava'


We use the `library` function (provided by the R system) multiple times to load specific libraries of functions.  The functions result in providing us all of the many functions contained in the libraries, which we can use later.

### Data Retrieval
___

In [ ]:
data = read.xlsx("../DATASETS/DATASET High-Tech Stocks.xlsx", sheetIndex=1, header=TRUE)
dim(data)
colnames(data)

To retrieve the dataset in file `dataset tech stocks.xlsx`, we use the `read.xlsx` function.  To confirm that the dataset has been retrieved and assigned, we use the `dim` and `colname` functions.

In [ ]:
head(data)

### Data Reformation
___

#### Cross-tabulate the 5 return variables by year and month.

In [ ]:
data$year = year(ymd(data$Calendar.Date))
data$month = month.abb[month(ymd(data$Calendar.Date))] # month.abb is already built into R system 
head(data)

In [ ]:
data.long = melt(data[,c(2:5,7,13:14)], id.vars=c("month","year"))
head(data.long)

To convert a table from one cross-tabulated form to another, we first add any new columns needed, convert it to "long" form (i.e., non-cross-tabulated form), and then convert it back to the cross-tabulated form that we want.<br>
<br>
We want the year and month for each row (i.e., the year and month at which each observation was made), so we need new columns for this information.<br>
<br>
To add a new `year` column to the `data` table, we assign a vector of year values to that column - the column will be created automatically.  To get the vector of year values, we convert the `Calendar.Date` column of values, using the `ymd` and `year` functions.  Taken all together, it looks like this:<br>
`data$year = year(ymd(data$Calendar.Date))`<br>
<br>
To add a new `month` column to the `data` table, we assign a vector of month values to that column - the column will be created automatically.  To get the vector of month values, we convert the `Calendar.Date` column of values, using the `ymd` and `months` functions, and the `month.abb` vector that's already provided by the R system.  Taken all together, it looks like this:<br>
`data$month = month.abb[month(ymd(data$Calendar.Date))]`<br>
<br>
To confirm that the new columns have been added, we use the `head` function to inspect the first few rows of the data.<br>
<br>
We use the `melt` function to convert a table to "long" form, where rows are indexed by year and month, the other original column headings are turned into categorical values and placed in a new `variable` column, and the original values are placed in a new `value` column.  The table we use comprises the original columns 2, 3, 4, 5, and 7, plus the new year and month columns 13 and 14.  We assign to the new "long" form table the new name `data.long`.  Note, the "long" form contains exactly the same information as the original form, just organized differently.  Taken all together, it looks like this:<br>
`data.long = melt(data[c(2:5,7,13:14)], id=c("month","year"))`<br>
<br>
To confirm that the form has been converted, we use the `head` function to inspect the first few rows of "long" form data.   

In [ ]:
dcast(data.long, variable ~ year, mean)

In [ ]:
dcast(data.long, variable ~ month, mean)

In [ ]:
dcast(data.long, variable ~ factor(month, month.abb), mean) # force months to be presented chronologically

In [ ]:
dcast(data.long, variable ~ year+month, mean)

In [ ]:
dcast(data.long, variable ~ year+factor(month, month.abb), mean) # force months to be presented chronologically

We use the `dcast` function to convert the "long" form table back into the cross-tabulated form that we want.<br>
<br>
To cross-tabulate as year vs. company (i.e., `year` column vs. `variable` column), with values aggregated by mean, we use the `dcast` function.  The 1st parameter is the "long" form table, in this case `data.long`.  The 2nd parameter is an indication of which columns to cross-tabulate (expressed as a formula), in this case `variable ~ year`.  The 3rd parameter is the aggreation function, in this case `mean`.  Note, the columns in the 2nd parameter are assumed to be associated with the table in the 1st parameter.  Note, `mean` is a function name, not a string, so it is not enclosed within <code>"&#8230;"</code>. Taken all together, it looks like this:<br>
`dcast(data.long, variable ~ year, mean)`<br>
<br>
To cross-tabulate as month vs. company, with values aggregated by mean, we again use the `dcast` function.  To force the months into chronological order, rather than alphabetical order, we use the `factor` function and the `month.abb` vector.  Note, in this cross-tabulation, a value corresponds to the mean of monthly values across a single month <b>name</b> across <b>all</b> years.  Taken all together, it looks like this:<br>
`dcast(data.long, variable ~ factor(month, month.abb), mean)`<br>
<br>
To cross-tabulate as year and month vs. company, with years organized by months, with values aggregated by mean, we again use the `dcast` function.  Taken all together it looks like this:<br>
`dcast(data.long, variable ~ year+factor(month, month.abb), mean)`<br>
<br>
<i>
About formulas:<br>
Some functions expect some of their parameters to be provided as formulae.  A formula is described by a column name, followed by the `~` symbol, followed by one or more other column names separated by `+` symbols, or followed by the `.` symbol meaning all other column names.  The column names are assumed to be associated with a specific table, usually provided as one of the function's other parameters.
</i>

#### Show the correlations between the 5 returns as a heat map. _(advanced)_

In [ ]:
m = cor(data[, c(2:5,7)])
m

In [ ]:
m.long = melt(m)
head(m.long)

In [ ]:
ggplot(m.long) +
geom_tile(aes(x=Var1, y=Var2, fill=value), color="white") +
geom_text(aes(x=Var1, y=Var2, label=sprintf("%0.2f", value)), size=2.5) +
scale_fill_gradient2(low="steelblue", mid="white", high="orangered", limits=c(-1,1)) +
xlab("") + ylab("")

To visualize all returns correlations as a heatmap, we use the `cor` function to compute the correlations, convert to "long" form, and then use the `geom_tile` and related functions.

`geom_tile` creates a layer of tiles. The 1st parameter is the tile locations and their colors.  We use the `aes` function to specify the tile locations and their colors.  Note, the `fill` parameter of the `aes` function takes `m.long`'s `value` column of values, which are the original correlation values, which also serve as the tile colors per a color scale (yet to be specified).  The `color` parameter is the tiles' edge color.

`geom_text` creates a layer of text labels for the tiles.  The 1st parameter is the text locations and the texts themselves.  We use the `aes` function to specify the text locations and the texts themselves.  Note, the `label` parameter of the `aes` function takes `m.long`'s `value` column of values, which are the original correlation values, formatted to 2 decimal places.  The `size` parameter is the texts' size.<br>
<br>
`scale_fill_gradient2` creates a layer specifying the color scale.

$\tiny \text{Copyright (c) Berkeley Data Analytics Group, LLC}$